<a href="https://colab.research.google.com/github/dawissem/fullpack-ia-module-1/blob/main/nlp_process_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Projet NLP : Analyse et Description d'Images
# À exécuter sur Google Colab

# Installation des bibliothèques nécessaires
!pip install transformers torch torchvision pillow gradio
!pip install --upgrade transformers

import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import base64

In [14]:


# Configuration du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé: {device}")

Device utilisé: cpu


In [15]:
class ImageDescriptionModel:
    def __init__(self):
        """Initialise le modèle de description d'images"""
        print("Chargement du modèle BLIP...")
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model.to(device)
        print("Modèle chargé avec succès!")

    def analyze_image(self, image):
        """Analyse une image et génère une description"""
        try:
            # Préparation de l'image
            if isinstance(image, str):
                # Si c'est un chemin de fichier
                image = Image.open(image).convert('RGB')
            elif isinstance(image, np.ndarray):
                # Si c'est un array numpy
                image = Image.fromarray(image).convert('RGB')

            # Traitement de l'image
            inputs = self.processor(image, return_tensors="pt").to(device)

            # Génération de la description
            with torch.no_grad():
                out = self.model.generate(**inputs, max_length=100, num_beams=5)

            # Décodage du texte
            description = self.processor.decode(out[0], skip_special_tokens=True)

            return description

        except Exception as e:
            return f"Erreur lors de l'analyse: {str(e)}"

    def analyze_with_context(self, image, context_prompt=""):
        """Analyse une image avec un contexte spécifique"""
        try:
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif isinstance(image, np.ndarray):
                image = Image.fromarray(image).convert('RGB')

            # Utilisation d'un prompt conditionnel si fourni
            if context_prompt:
                inputs = self.processor(image, context_prompt, return_tensors="pt").to(device)
            else:
                inputs = self.processor(image, return_tensors="pt").to(device)

            with torch.no_grad():
                out = self.model.generate(**inputs, max_length=150, num_beams=5)

            description = self.processor.decode(out[0], skip_special_tokens=True)

            # Nettoyer la description si elle contient le prompt
            if context_prompt and description.startswith(context_prompt):
                description = description[len(context_prompt):].strip()

            return description

        except Exception as e:
            return f"Erreur lors de l'analyse contextuelle: {str(e)}"

In [16]:

class ImageAnalyzer:
    def __init__(self):
        """Initialise l'analyseur d'images"""
        self.model = ImageDescriptionModel()

    def full_analysis(self, image, include_context=True):
        """Effectue une analyse complète de l'image"""
        results = {
            "description_generale": "",
            "description_detaillee": "",
            "elements_detectes": "",
            "couleurs_dominantes": "",
            "style_artistic": ""
        }

        try:
            # Description générale
            results["description_generale"] = self.model.analyze_image(image)

            if include_context:
                # Description détaillée
                results["description_detaillee"] = self.model.analyze_with_context(
                    image, "Décris cette image en détail:"
                )

                # Éléments détectés
                results["elements_detectes"] = self.model.analyze_with_context(
                    image, "Quels objets vois-tu dans cette image?"
                )

                # Analyse des couleurs
                results["couleurs_dominantes"] = self.analyze_colors(image)

                # Style artistique
                results["style_artistic"] = self.model.analyze_with_context(
                    image, "Quel est le style artistique de cette image?"
                )

            return results

        except Exception as e:
            return {"erreur": f"Erreur lors de l'analyse complète: {str(e)}"}

    def analyze_colors(self, image):
        """Analyse les couleurs dominantes de l'image"""
        try:
            if isinstance(image, str):
                img = Image.open(image).convert('RGB')
            elif isinstance(image, np.ndarray):
                img = Image.fromarray(image).convert('RGB')
            else:
                img = image

            # Redimensionner pour l'analyse
            img = img.resize((150, 150))

            # Convertir en array numpy
            img_array = np.array(img)

            # Calculer les couleurs moyennes
            avg_color = np.mean(img_array, axis=(0, 1))

            # Déterminer les couleurs dominantes
            colors = []
            if avg_color[0] > avg_color[1] and avg_color[0] > avg_color[2]:
                colors.append("rouge")
            if avg_color[1] > avg_color[0] and avg_color[1] > avg_color[2]:
                colors.append("vert")
            if avg_color[2] > avg_color[0] and avg_color[2] > avg_color[1]:
                colors.append("bleu")

            if not colors:
                if np.mean(avg_color) > 200:
                    colors.append("clair")
                elif np.mean(avg_color) < 100:
                    colors.append("sombre")
                else:
                    colors.append("neutre")

            return f"Couleurs dominantes: {', '.join(colors)}"

        except Exception as e:
            return f"Erreur analyse couleurs: {str(e)}"



In [ ]:
def create_gradio_interface():
    """Crée l'interface Gradio pour l'application"""

    # Initialiser l'analyseur
    analyzer = ImageAnalyzer()

    def process_image(image, analysis_type, custom_prompt=""):
        """Traite l'image selon le type d'analyse demandé"""
        if image is None:
            return "Veuillez télécharger une image."

        try:
            if analysis_type == "Description simple":
                result = analyzer.model.analyze_image(image)
                return f"Description: {result}"

            elif analysis_type == "Description avec contexte":
                if custom_prompt:
                    result = analyzer.model.analyze_with_context(image, custom_prompt)
                    return f"Description contextuelle: {result}"
                else:
                    return "Veuillez fournir un prompt pour l'analyse contextuelle."

            elif analysis_type == "Analyse complète":
                results = analyzer.full_analysis(image)

                output = "=== ANALYSE COMPLÈTE ===\n\n"
                for key, value in results.items():
                    if key != "erreur":
                        output += f"{key.replace('_', ' ').title()}: {value}\n\n"

                return output

        except Exception as e:
            return f"Erreur lors du traitement: {str(e)}"

    # Interface Gradio
    with gr.Blocks(title="Analyseur d'Images NLP", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🖼️ Analyseur d'Images avec NLP")
        gr.Markdown("Téléchargez une image et obtenez une description détaillée générée par IA.")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(
                    label="Téléchargez votre image",
                    type="pil"
                )

                analysis_type = gr.Dropdown(
                    choices=["Description simple", "Description avec contexte", "Analyse complète"],
                    value="Description simple",
                    label="Type d'analyse"
                )

                custom_prompt = gr.Textbox(
                    label="Prompt personnalisé (optionnel)",
                    placeholder="Ex: Décris les émotions dans cette image",
                    visible=False
                )

                analyze_btn = gr.Button("Analyser l'image", variant="primary")

                # Montrer/cacher le prompt personnalisé
                def toggle_prompt(choice):
                    return gr.update(visible=(choice == "Description avec contexte"))

                analysis_type.change(toggle_prompt, analysis_type, custom_prompt)

            with gr.Column():
                output_text = gr.Textbox(
                    label="Résultat de l'analyse",
                    lines=15,
                    max_lines=20
                )

        # Exemples d'images
        gr.Markdown("### Exemples d'images à tester:")
        example_images = [
            ["https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=400"],
            ["https://images.unsplash.com/photo-1551963831-b3b1ca40c98e?w=400"],
            ["https://images.unsplash.com/photo-1518717758536-85ae29035b6d?w=400"]
        ]

        gr.Examples(
            examples=example_images,
            inputs=image_input,
            label="Cliquez sur une image d'exemple"
        )

        # Connecter les événements
        analyze_btn.click(
            process_image,
            inputs=[image_input, analysis_type, custom_prompt],
            outputs=output_text
        )

    return demo

# Fonction principale
def main():
    """Fonction principale pour lancer l'application"""
    print("Initialisation de l'application...")

    # Créer et lancer l'interface
    demo = create_gradio_interface()

    print("Lancement de l'interface Gradio...")
    demo.launch(
        share=True,  # Permet de partager l'interface
        debug=True,
        server_name="0.0.0.0",
        server_port=7860
    )

# Instructions d'utilisation
print("""
=== INSTRUCTIONS D'UTILISATION ===

1. Exécutez toutes les cellules dans l'ordre
2. Attendez que les modèles se chargent
3. Utilisez l'interface Gradio qui s'ouvre
4. Téléchargez une image et choisissez le type d'analyse
5. Cliquez sur "Analyser l'image"

Types d'analyse disponibles:
- Description simple: Description générale de l'image
- Description avec contexte: Description avec un prompt personnalisé
- Analyse complète: Analyse détaillée avec plusieurs aspects

Pour lancer l'application, exécutez:
main()
""")

# Décommenter la ligne suivante pour lancer automatiquement
main()


=== INSTRUCTIONS D'UTILISATION ===

1. Exécutez toutes les cellules dans l'ordre
2. Attendez que les modèles se chargent
3. Utilisez l'interface Gradio qui s'ouvre
4. Téléchargez une image et choisissez le type d'analyse
5. Cliquez sur "Analyser l'image"

Types d'analyse disponibles:
- Description simple: Description générale de l'image
- Description avec contexte: Description avec un prompt personnalisé
- Analyse complète: Analyse détaillée avec plusieurs aspects

Pour lancer l'application, exécutez:
main()

Initialisation de l'application...
Chargement du modèle BLIP...
Modèle chargé avec succès!
Lancement de l'interface Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aad3a6f417e7a7311f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to H

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3570: UserWarning: image file could not be identified because AVIF support not installed
  warnings.warn(message)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2187, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1853, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "